# Importing shapefiles into GOSTnets

Testing a modification of the load_osm file, not yet committed to the GOSTNets repo

In [4]:
import os, sys, importlib

import pandas as pd
import geopandas as gpd
import networkx as nx

from shapely.wkt import loads
from shapely.geometry import Point
from random import sample 

## Your GOSTnets path could be different
#sys.path.append(r"C:\Work\Code\GOSTnets")
sys.path.append("../../../GOSTNets")
import GOSTnets as gn

In [5]:
from GOSTnets.load_osm import *

In [6]:
# turn on extention to auto reload module changes
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# KDTree test
from scipy import spatial

In [8]:
A = np.random.random((9000,2))*1000

In [9]:
A

array([[841.11858117, 936.06417174],
       [488.6504773 , 215.47755373],
       [181.39856247, 573.30666564],
       ...,
       [463.87004062, 576.60092587],
       [334.39792255, 181.94625815],
       [424.93348988, 660.06104295]])

In [10]:
#pt = [6, 30]
#A[spatial.KDTree(A).query(pt)[1]]
reg_c_start = time.time()
print("reg_c_start start")
A_KD_tree = spatial.KDTree(A)
reg_c_end = time.time()
print(reg_c_end - reg_c_start)

reg_c_start start
0.03467392921447754


In [11]:
reg_c_start = time.time()
print("reg_c_start start")
distances, indices = A_KD_tree.query(A)
reg_c_end = time.time()
print(reg_c_end - reg_c_start)

reg_c_start start
1.206179141998291


In [12]:
reg_c_start = time.time()
print("reg_c_start start")
A_cKD_tree = spatial.cKDTree(A)
reg_c_end = time.time()
print(reg_c_end - reg_c_start)

reg_c_start start
0.002904176712036133


In [13]:
reg_c_start = time.time()
print("reg_c_start start")
distances, indices = A_cKD_tree.query(A)
reg_c_end = time.time()
print(reg_c_end - reg_c_start)

reg_c_start start
0.010185956954956055


In [14]:
# read back your graph from step 1 from you saved pickle
G = nx.read_gpickle('cxb_roads_unclean.pickle')

#Iceland_UTMZ = {'init': 'epsg:32627'}

#WGS = {'init': 'epsg:4326'} # do not adjust. OSM natively comes in ESPG 4326

In [15]:
list(G.nodes(data=True))

[(0, {'x': 92.0836204, 'y': 21.4308841}),
 (1, {'x': 92.0889015, 'y': 21.3659021}),
 (2, {'x': 92.07222, 'y': 21.7011036}),
 (3, {'x': 92.1314013, 'y': 21.291856}),
 (4, {'x': 92.1400112, 'y': 21.269837}),
 (5, {'x': 92.0990773, 'y': 21.2696527}),
 (6, {'x': 92.1636749, 'y': 21.1385552}),
 (7, {'x': 92.0860072, 'y': 21.472696}),
 (8, {'x': 92.1437672, 'y': 21.172236}),
 (9, {'x': 92.1467223, 'y': 21.2118446}),
 (10, {'x': 92.1310231, 'y': 21.3494821}),
 (11, {'x': 92.2456088, 'y': 20.9720141}),
 (12, {'x': 92.299682, 'y': 20.8350753}),
 (13, {'x': 92.2982658, 'y': 20.8675693}),
 (14, {'x': 92.294654, 'y': 20.820628}),
 (15, {'x': 92.295072, 'y': 20.8377033}),
 (16, {'x': 92.2995215, 'y': 20.8341894}),
 (17, {'x': 92.2444443, 'y': 20.9707092}),
 (18, {'x': 92.2494361, 'y': 20.9705359}),
 (19, {'x': 92.0199647, 'y': 21.4229312}),
 (20, {'x': 92.1545878, 'y': 21.1645294}),
 (21, {'x': 92.0334731, 'y': 21.7053116}),
 (22, {'x': 92.2203938, 'y': 21.0748258}),
 (23, {'x': 92.0721463, 'y': 21

In [16]:
'''
# the nodes in the dataset do not have coordinates, so let's fix that
edges = list(G.edges(data=True))
nodes = G.nodes(data=True)
all_nodes = []
# Loop through all the nodes to extract their coordinates from the nodes
for n in nodes:
    # For the current node, loop through the edges until we find an edge witht he current node
    found_node = False
    edge_count = 0
    while not found_node:
        e = edges[edge_count]
        edge_count = edge_count + 1        
        # if the current node is part of the current edge, we can extract the coordinate        
        if n[0] in e: 
            found_node = True
            # The coordinate for the node is either the first or final coordinate of the current edge            
            pt_idx = 0
            if e.index(n[0]) == 1:
                pt_idx = -1
            #Extract the appropriate point and store the new node
            pt = list(e[2]['geometry'].coords)[pt_idx]
            node_vals = {'x':pt[0], 'y':pt[1]}
            all_nodes.append([n[0], node_vals])
            #G.remove_node(n[0])
            #G.add_node(n[0], **node_vals)
G.update(nodes=all_nodes)
'''

"\n# the nodes in the dataset do not have coordinates, so let's fix that\nedges = list(G.edges(data=True))\nnodes = G.nodes(data=True)\nall_nodes = []\n# Loop through all the nodes to extract their coordinates from the nodes\nfor n in nodes:\n    # For the current node, loop through the edges until we find an edge witht he current node\n    found_node = False\n    edge_count = 0\n    while not found_node:\n        e = edges[edge_count]\n        edge_count = edge_count + 1        \n        # if the current node is part of the current edge, we can extract the coordinate        \n        if n[0] in e: \n            found_node = True\n            # The coordinate for the node is either the first or final coordinate of the current edge            \n            pt_idx = 0\n            if e.index(n[0]) == 1:\n                pt_idx = -1\n            #Extract the appropriate point and store the new node\n            pt = list(e[2]['geometry'].coords)[pt_idx]\n            node_vals = {'x':pt[0]

In [17]:
# inspect the resulting Graph
nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
print(len(nodes))
print(nodes[0])
print(len(edges))
print(edges[0])

7929
(0, {'x': 92.0836204, 'y': 21.4308841})
18277
(0, 3527, {'Wkt': <shapely.geometry.linestring.LineString object at 0x120e6aa10>, 'id': 2, 'infra_type': 'unclassified', 'osm_id': 1, 'key': 'edge_2', 'length': 0.4669350413713456})


In [18]:
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [19]:
#list_of_graphs[1].number_of_edges()

In [20]:

print('print G before')
print(G.size())
# save the largest subgraph for networkX 2.4
largest = max(nx.strongly_connected_components(G), key=len)
G = nx.induced_subgraph(G,largest).copy()
print('print G after')
print(G.size())


print G before
18277
print G after
382


In [21]:
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [22]:
#gn.save(G,'G_largest_graph','./',pickle=False,nodes=False,edges=True)

In [23]:
#nx.strongly_connected_components(G)

In [24]:
# Identify only the largest graph

#list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))
'''
list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.strongly_connected_components(G))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i
G = max_graph
'''

'\nlist_of_subgraphs = list(G.subgraph(c).copy() for c in nx.strongly_connected_components(G))\nmax_graph = None\nmax_edges = 0\nfor i in list_of_subgraphs:\n    if i.number_of_edges() > max_edges:\n        max_edges = i.number_of_edges()\n        max_graph = i\nG = max_graph\n'

In [25]:
#gn.save(G,'G_largest_graph2','./',pickle=False,nodes=False,edges=True)

In [26]:
# inspect the resulting Graph
nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
print(len(nodes))
print(nodes[0])
print(len(edges))
print(edges[0])

163
(1024, {'x': 92.0504168, 'y': 21.4105411})
382
(1024, 2851, {'Wkt': <shapely.geometry.linestring.LineString object at 0x129024ed0>, 'id': 1539, 'infra_type': 'secondary', 'osm_id': 2369, 'key': 'edge_1539', 'length': 0.4786572763506625})


In [27]:
# read in origins and destinations
#input_folder = r"C:\Work\Code\GOST_PublicGoods\Implementations\RobertBanick_ShapefileTToG\SampleData"
input_folder = r"./input_data"
origins_file = os.path.join(input_folder, 'hrsl_pts_admins.csv')
destinations_file  = os.path.join(input_folder, 'education_cxb/education_cxb_lged.shp')

inO = pd.read_csv(origins_file)
inD = gpd.read_file(destinations_file)
inD = inD.to_crs({'init': 'epsg:4326'})

/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [28]:
len(inO)

120498

In [29]:
inO

,hrsl_ID,VALUE,geometry,Upaz_name,Upz_UID,index_right,ADM4_EN,ADM4_PCODE
0,0,20.187097,MULTIPOINT (91.8750000000112 21.91583333332886),Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581
1,3,20.187097,MULTIPOINT (91.87527777778898 21.91499999999552),Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581
2,4,20.187097,MULTIPOINT (91.87583333334454 21.91472222221774),Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581
3,5,20.187097,MULTIPOINT (91.87666666667789 21.91472222221774),Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581
4,6,20.187097,MULTIPOINT (91.87722222223346 21.91472222221774),Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581
...,...,...,...,...,...,...,...,...
120493,122702,38.083674,MULTIPOINT (92.32833333334332 20.59722222221616),Teknaf Upazila,202290,4562,St.Martin Dwip,20229039
120494,122707,38.083674,MULTIPOINT (92.3302777777878 20.59638888888282),Teknaf Upazila,202290,4562,St.Martin Dwip,20229039
120495,122709,38.083674,MULTIPOINT (92.3302777777878 20.59611111110505),Teknaf Upazila,202290,4562,St.Martin Dwip,20229039
120496,122713,38.083674,MULTIPOINT (92.33111111112115 20.59305555554948),Teknaf Upazila,202290,4562,St.Martin Dwip,20229039


In [30]:
#Convert the input origins to a POINT geodaframe
inO_geom = [loads(x) for x in inO['geometry']]
inO_geom = [Point(x[0].x, x[0].y) for x in inO_geom]

inO = gpd.GeoDataFrame(inO.drop(['geometry'], axis=1), geometry = inO_geom, crs = {'init': 'epsg:4326'})

# sample origins for testing
# inO = inO.loc[sample(list(inO.index), 1000)]

In [31]:
print(inO.shape)
print(inO.crs)
print(inD.shape)
print(inD.crs)

(120498, 8)
{'init': 'epsg:4326'}
(788, 20)
{'init': 'epsg:4326'}


In [32]:
inO[:100]

,hrsl_ID,VALUE,Upaz_name,Upz_UID,index_right,ADM4_EN,ADM4_PCODE,geometry
0,0,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87500 21.91583)
1,3,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87528 21.91500)
2,4,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87583 21.91472)
3,5,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87667 21.91472)
4,6,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87722 21.91472)
...,...,...,...,...,...,...,...,...
95,122,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.88500 21.90750)
96,123,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.88583 21.90750)
97,124,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.88611 21.90750)
98,127,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.86278 21.90694)


In [33]:
in0_subset = inO[:100]

In [34]:
in0_subset

,hrsl_ID,VALUE,Upaz_name,Upz_UID,index_right,ADM4_EN,ADM4_PCODE,geometry
0,0,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87500 21.91583)
1,3,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87528 21.91500)
2,4,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87583 21.91472)
3,5,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87667 21.91472)
4,6,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.87722 21.91472)
...,...,...,...,...,...,...,...,...
95,122,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.88500 21.90750)
96,123,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.88583 21.90750)
97,124,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.88611 21.90750)
98,127,20.187097,Kutubdia Upazila,202245,4815,Uttar Dhurung,20224581,POINT (91.86278 21.90694)


In [35]:
# time it
import time

start = time.time()
print("start")
origins_subset = gn.pandana_snap(G, in0_subset)
end = time.time()
print(end - start)

start


/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


13.231512784957886


In [36]:
#import cython

In [37]:
# test ver 2
start = time.time()
print("start")
origins_subset2 = gn.pandana_snap_c(G, in0_subset)
end = time.time()
print(end - start)

start
print in_df
    hrsl_ID      VALUE         Upaz_name  Upz_UID  index_right        ADM4_EN  \
0         0  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
1         3  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
2         4  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
3         5  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
4         6  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
..      ...        ...               ...      ...          ...            ...   
95      122  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
96      123  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
97      124  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
98      127  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   
99      128  20.187097  Kutubdia Upazila   202245         4815  Uttar Dhurung   

    ADM4_

/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


UnboundLocalError: local variable 'func_start' referenced before assignment

In [43]:
# test ver 3
origins_subset2 = gn.pandana_snap_c(G, in0_subset, time_it = True)

not reprojecting
time elapsed for function
0.016115188598632812


In [ ]:
origins = gn.pandana_snap(G, inO)

In [ ]:
destinations = gn.pandana_snap(G, inD)

In [ ]:
origins.loc[origins['NN'] == 3140]

In [ ]:
destinations

In [ ]:
oNodes = origins['NN'].unique()
dNodes = destinations['NN'].unique()

In [ ]:
od = gn.calculate_OD(G, oNodes, dNodes, -1)
od.shape

In [ ]:
od

In [ ]:
OD_df = pd.DataFrame(od, columns = dNodes, index = oNodes)

In [ ]:
OD_df[:500]

In [ ]:
OD_df.to_csv("./output_csv2.csv")

In [ ]:
'''# Write the largest graph back to file
all_data = {}
idx=0
for o, d, data in G.edges(data=True):
    data['o'] = o
    data['d'] = d
    all_data[idx] = data    
    idx = idx+1
big_graph = gpd.GeoDataFrame(all_data).transpose()
big_graph.to_csv(input_shapefile.replace(".shp", "_big_boy.csv"))
big_graph.head()'''